In [1]:
using Images
using FFTW
using Formatting

In [2]:
LUMINANCE_QUANTIZATION_TABLE = [
    16  11  10  16  24   40   51   61   
    12  12  14  19  26   58   60   55   
    14  13  16  24  40   57   69   56   
    14  17  22  29  51   87   80   62   
    18  22  37  56  68   109  103  77   
    24  35  55  64  81   104  113  92   
    49  64  78  87  103  121  120  101  
    72  92  95  98  112  100  103  99   
  ]

ZIGZAG = [
    0   1   5   6  14  15  27  28
    2   4   7  13  16  26  29  42
    3   8  12  17  25  30  41  43
    9  11  18  24  31  40  44  53
    10  19  23  32  39  45  52  54
    20  22  33  38  46  51  55  60
    21  34  37  47  50  56  59  61
    35  36  48  49  57  58  62  63
   ] .+ 1

8×8 Array{Int64,2}:
  1   2   6   7  15  16  28  29
  3   5   8  14  17  27  30  43
  4   9  13  18  26  31  42  44
 10  12  19  25  32  41  45  54
 11  20  24  33  40  46  53  55
 21  23  34  39  47  52  56  61
 22  35  38  48  51  57  60  62
 36  37  49  50  58  59  63  64

In [3]:
function to_zigzag(block)
  out_block = zeros(eltype(block), size(ZIGZAG))
  for idx in CartesianIndices(ZIGZAG)
    out_block[ZIGZAG[idx]] = block[idx]
    # display(out_block)
  end
  reshape(out_block, (1,64))
end

to_zigzag (generic function with 1 method)

In [25]:
img = load("../images/f124074.jpg")
img_ycbcr = channelview(YCbCr.(img))

for c in 1:3
    channel = img_ycbcr[c,:,:]
    row, col = size(channel)

    display(channel)
    
    DC = []
    AC = []

    for i in 1:8:row
      for j in 1:8:col
        block = channel[i:i+7, j:j+7]

        block_shifted = block .- 127
        block_dct = FFTW.dct(block_shifted)

        block_quantization = round.(Int32, block_dct ./ LUMINANCE_QUANTIZATION_TABLE)
        # block_quantization = block_dct ./ LUMINANCE_QUANTIZATION_TABLE
        block_zigzag = to_zigzag(block_quantization)

        if c == 1 && i == 1 + 8*10 && j == 1 + 8*3
          display(block)
          display(block_shifted)
          display(block_dct)
          display(block_quantization)
          display(block_zigzag)
        end

        # block_zigzag = collect(block_zigzag)
        push!(DC, block_zigzag[1])
        push!(AC, block_zigzag[2:end])
      end
    end

    if c == 1
        display(DC)
        display(AC)
    end
end


720×1280 Array{Float32,2}:
 39.2166  39.4639  39.4639  39.0577  …  37.4843  37.4843  37.4843  37.4843
 39.4639  39.4639  39.0577  39.0577     37.4843  37.4843  37.4843  37.4843
 39.4639  39.0577  39.0577  39.0577     37.4843  37.4843  37.4843  37.4843
 39.0577  39.0577  39.0577  39.9165     38.0348  37.4843  37.4843  37.4843
 39.0577  39.9165  39.9165  39.9165     38.0348  37.176   37.4843  37.4843
 39.9165  39.9165  39.9165  40.1639  …  38.0348  38.0348  37.4843  37.4843
 39.9165  39.9165  39.9165  40.1639     38.0348  38.0348  37.4843  37.4843
 39.9165  39.9165  39.9165  40.1639     38.1327  38.0348  38.3432  37.4843
 39.4639  39.4639  39.9165  40.1639     39.4493  38.3432  38.0348  38.5484
 39.4639  39.4639  39.0577  40.1639     39.4493  38.3432  38.0348  38.5484
 39.4639  39.4639  39.0577  40.1639  …  39.4493  38.5905  38.0348  38.0348
 39.4639  39.0577  39.0577  40.1639     39.1925  39.4493  38.5905  38.0348
 39.0577  39.0577  39.0577  40.1639     39.1925  39.4493  38.5905  38.590

8×8 Array{Float32,2}:
 55.4277  55.736   54.8772  54.8772  55.736   55.736   55.736   55.736 
 56.2866  56.2866  57.4537  57.4537  56.5949  55.736   55.736   55.736 
 57.9948  57.9948  57.1359  56.0813  55.2225  55.2225  55.736   56.5949
 58.2421  58.2421  58.2421  58.2421  58.2421  58.0463  56.9401  56.9401
 57.3833  58.4894  61.0659  62.7836  63.6424  61.6774  57.7989  55.2225
 58.4894  58.4894  58.9326  60.6502  61.5091  61.0659  58.2421  56.0813
 61.0659  59.6935  59.7914  59.1799  58.9326  58.0832  58.2421  57.7989
 61.0659  62.27    64.0855  64.0246  62.3679  59.8009  58.2421  56.9401

8×8 Array{Float32,2}:
 -71.5723  -71.264   -72.1228  -72.1228  …  -71.264   -71.264   -71.264 
 -70.7134  -70.7134  -69.5463  -69.5463     -71.264   -71.264   -71.264 
 -69.0052  -69.0052  -69.8641  -70.9187     -71.7775  -71.264   -70.4051
 -68.7579  -68.7579  -68.7579  -68.7579     -68.9537  -70.0599  -70.0599
 -69.6167  -68.5106  -65.9341  -64.2164     -65.3226  -69.2011  -71.7775
 -68.5106  -68.5106  -68.0674  -66.3498  …  -65.9341  -68.7579  -70.9187
 -65.9341  -67.3065  -67.2086  -67.8201     -68.9168  -68.7579  -69.2011
 -65.9341  -64.73    -62.9145  -62.9754     -67.1991  -68.7579  -70.0599

8×8 Array{Float32,2}:
 -550.578        4.7784      …   0.00653681   0.205445    0.212533 
  -13.5756      -2.92463        -0.472033    -0.130214   -0.218505 
   -1.48771      1.21404         0.279535    -0.391432    0.330444 
   -0.673342    -2.97597        -0.222817    -0.0539551  -0.0560064
    3.04324      0.102968       -0.474424    -0.0202312   0.0131407
   -3.61937     -0.509858    …  -0.253103    -0.123185   -0.0314786
   -0.353786     0.00731296     -0.460769    -0.278703   -0.466107 
   -0.00664318   0.52792        -0.0775939   -0.022749    0.307393 

8×8 Array{Int32,2}:
 -34  0  -1  0  0  0  0  0
  -1  0   0  0  0  0  0  0
   0  0   0  0  0  0  0  0
   0  0   0  0  0  0  0  0
   0  0   0  0  0  0  0  0
   0  0   0  0  0  0  0  0
   0  0   0  0  0  0  0  0
   0  0   0  0  0  0  0  0

1×64 Array{Int32,2}:
 -34  0  -1  0  0  -1  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0

14400-element Array{Any,1}:
 -44
 -43
 -43
 -42
 -42
 -42
 -41
 -41
 -41
 -41
 -40
 -40
 -39
   ⋮
 -42
 -41
 -41
 -41
 -41
 -42
 -43
 -43
 -44
 -44
 -44
 -45

14400-element Array{Any,1}:
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 Int32[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
 ⋮                                               

720×1280 Array{Float32,2}:
 141.368  141.226  141.226  141.956  …  143.855  143.855  143.855  143.855
 141.226  141.226  141.956  141.956     143.855  143.855  143.855  143.855
 141.226  141.956  141.956  141.956     143.855  143.855  143.855  143.855
 141.956  141.956  141.956  141.956     145.025  143.855  143.855  143.855
 141.956  141.956  141.956  141.956     145.025  145.025  143.855  143.855
 141.956  141.956  141.956  141.813  …  145.025  145.025  143.855  143.855
 141.956  141.956  141.956  141.813     145.025  145.025  143.855  143.855
 141.956  141.956  141.956  141.813     145.464  145.025  143.855  143.855
 141.226  141.226  141.956  141.813     143.713  143.855  145.025  144.728
 141.226  141.226  141.956  141.813     143.713  143.855  145.025  144.728
 141.226  141.226  141.956  141.813  …  143.713  143.713  145.025  145.025
 141.226  141.956  141.956  141.813     143.861  143.713  143.713  145.025
 141.956  141.956  141.956  141.813     143.861  143.713  143.713  143.71

720×1280 Array{Float32,2}:
 115.448  114.641  114.641  114.938  …  115.459  115.459  115.459  115.459
 114.641  114.641  114.938  114.938     115.459  115.459  115.459  115.459
 114.641  114.938  114.938  114.938     115.459  115.459  115.459  115.459
 114.938  114.938  114.938  114.938     115.684  115.459  115.459  115.459
 114.938  114.938  114.938  114.938     115.684  115.684  115.459  115.459
 114.938  114.938  114.938  114.131  …  115.684  115.684  115.459  115.459
 114.938  114.938  114.938  114.131     115.684  115.684  115.459  115.459
 114.938  114.938  114.938  114.131     115.612  115.684  115.459  115.459
 114.641  114.641  114.938  114.131     114.652  115.459  115.684  116.562
 114.641  114.641  114.938  114.131     114.652  115.459  115.684  116.562
 114.641  114.641  114.938  114.131  …  114.652  114.652  115.684  115.684
 114.641  114.938  114.938  114.131     114.213  114.652  114.652  115.684
 114.938  114.938  114.938  114.131     114.213  114.652  114.652  114.65